In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from subprocess import check_output
# print(check_output(["ls", ""]).decode("utf8"))

# any results you write to the current directory are saved as output

## load train and test data

In [ ]:
# create and training & test sets, skipping the header row with [1:]
train = pd.read_csv("")
print(train.shape)
train.head()

In [ ]:
X_train = (train.iloc[:, 1:].values).astype("float32")
# cast a dataframe object to specified data type

# all pixel values
y_train = train.iloc[:, 0].values.astype('int32')

# only labels i.e target digits
X_test = test.values.astype('float32')

In [ ]:
X_train

In [ ]:
y_train

## preprocessing the digit images

### feature standardization

In [ ]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

def standardize(x):
    return (x - mean_px)/std_px

### one hot encoding of labels

In [ ]:
from keras.utils.np_utls import to_categorical
y_train = to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

In [ ]:
plt.title(y_train[9])
plt.plot(y_train[9])
plt.xticks(range[10])

## designing neural network architecture

In [ ]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

### linear model

In [ ]:
from keras.models import Sequential
from keras.layers.core import Lambda, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D
'''
create a simple model from Keras Sequential layer

1. lambda layer performs simple arithmetic operations like sum, average, exponentiation etc
    In 1st layer performs simple arithmetic operations like sum, average, exponentiation etc
2. flatten will transform input into 1D array
3. dense is fully connected layer that means all neurons in previous layers will be connected to all neurons
    in connected layer. In the last layer we have to specify output dimensions/classes of the model. 
    Here it is 10, since we have to output 10 different digit labels
'''

In [ ]:
model = Sequential()
model.add(Lambda(standardize, input_shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
print("input shape", model.input_shape)
print("output shape", model.output_shape)

## compile network

In [ ]:
'''
before making the network ready for training we have to make sure to add below things
    1.a loss function: to measure how good the network is
    2.an optimizer: to update network as it sees more data and reduce loss value
    3.metrics: to monitor performance of network
'''

In [ ]:
from keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.preprocessing import image
gen = image.ImageDataGenerator()

### cross validation

In [ ]:
from sklearn.model_selection import train_test_split
X = X_train
y = y_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
batches = gen.flow(X_train, y_train, batch_size=64)
val_batches=gen.flow(X_val, y_val, batch_size=64)

In [ ]:
history = model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=3,
                             validation_data=val_batches, validation_steps=val_batches.n)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, 'bo')

plt.plot(epochs, val_loss_values, 'b+')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc_values, 'bo')
plt.plot(epochs, val_acc_values, 'b+')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
s
plt.show()

## fully connected model

neurons is a fully connected layer have full connections to all activations in the previous layer, as seen in regular Neural Networks. Adding another Dense layer to model

In [ ]:
def get_fc_model():
    model = Sequential([Lambda(standardize, input_shape = (28, 28, 1)),
                       Flatten(),
                       Dense(512, activation='relu'),
                       Dense(10, activation='softmax')
                       ])
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc = get_fc_model()
fc.optimizer.lr = 0.01

In [ ]:
history = fc.fit_generator(generator=batches,
                          steps_per_epoch=batches.n,
                          epochs=1,
                          validation_data=val_batches,
                          validation_steps=val_batches.n)

## convolutional neural network

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D

def get_cnn_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28, 28, 1)),
        Convolution2D(32, (3, 3), activation='relu'),
        Convolution2D(32, (3, 3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64, (3, 3), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = get_cnn_model()
model.optimizer.lr = 0.01

In [ ]:
history = model.fit_generator(generator=batches, steps_per_epoch=batches.n,
                             epochs=1,
                             validation_data=val_batches,
                             validation_steps=val_batches.n)


## data augmentation

In [ ]:
'''
it is technique of showing slightly different of new images to neural network to avoid overfitting.
and to achieve better generalization.
in case you have very small dataset, you can use different kinds of data augmentation techiniques to
increase your data size.
neural networks perform better if you provide them with more data
different data augmentation techniques are as follows:
1.cropping
2.rotating
3.scaling
4.translating
5.flipping
6.adding gaussian noise to input images etc.
'''

In [ ]:
gen =ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
val_batches = gen.flow(X_val, y_val, batch_size=64)

In [ ]:
model.optimizer.lr=0.001
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                    validation_data=val_batches, validation_steps=val_batches.n)

## adding batch normalization

BN helps to fine tune hyperparameters move better and train really deep neural networks

In [ ]:
from keras.layers.normalization import BatchNormalization

def get_bn_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model= get_bn_model()
model.optimizer.lr=0.01
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                    validation_data=val_batches, validation_steps=val_batches.n)